# Dependencies

In [ ]:
import pandas as pd
import requests
import json
import numpy as np
import matplotlib.pyplot as plt

import datetime as dt
import scipy.stats as sts
import scipy.ndimage as ndimage


from math import isnan
from matplotlib.animation import FuncAnimation
from matplotlib import animation, rc
from Resources.config import app_token

# Additional Depedencies which requires updating your environment

In [ ]:
import descartes
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon

# Import Resources

In [ ]:
## Source: Crimes - 2001 to present
## https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2
url_crime_data = "https://data.cityofchicago.org/resource/ijzp-q8t2.json"
date_filter="$WHERE=DATE BETWEEN '2014-01-01' AND '2018-12-31'"
crime_records_limit = 1000
## Total # 1344565
url_crime_data_with_filter = f"{url_crime_data}?{date_filter}&$limit={crime_records_limit}&$$app_token={app_token}"
crime_data_json = requests.get(url_crime_data_with_filter).json()
## Convert to pandas DataFrame
crime_data_df = pd.DataFrame.from_records(crime_data_json)
## drop extra columns. Optional
#crime_data_df = crime_data_df.drop(['x_coordinate','y_coordinate','latitude','longitude','location'],axis=1)

In [ ]:
# Street Light Data Source from 2014-2018
street_light_reports = "Resources/Service_Requests_Street_Lights.csv" 

# Read Street Light Data and store into Pandas data frame, add ability to read special chrarters
street_light_reports_pd = pd.read_csv(street_light_reports,encoding= "ISO-8859-1")

In [ ]:
# Community Area Names Data Source 
community_area_names = "Resources/CommAreas_20200108.csv" 

# Read Community Area Names Data and store into Pandas data frame, add ability to read special characters
community_area_names_pd = pd.read_csv(community_area_names,encoding= "ISO-8859-1")

In [ ]:
# Community Area Names Data Source 
pop_census_data = "Resources/Pop-Census-Data-by-Chicago-Community-Area-2017.csv" 

# Read Pop Census Data by Chicago Community and store into Pandas data frame.
pop_census_data_pd = pd.read_csv(pop_census_data)

# Data cleanup

In [ ]:
#Data Cleanup - Rename Creation Date Column Because it Has Unnecessary Charaters in the CSV Field
street_light_reports_pd = street_light_reports_pd.rename(columns={'ÈÀCreation Date': 'Creation Date'})

# Add a Column for Year, Month, Day in Street Light Outage Data 
street_light_reports_pd['Year'] = pd.DatetimeIndex(street_light_reports_pd['Creation Date']).year
street_light_reports_pd['Month'] = pd.DatetimeIndex(street_light_reports_pd['Creation Date']).month
street_light_reports_pd['Day'] = pd.DatetimeIndex(street_light_reports_pd['Creation Date']).day

#Filter Street Light data for everything greater than 2013 and less than 2018
street_light_reports_pd = street_light_reports_pd.loc[(street_light_reports_pd['Year']>2013) & (street_light_reports_pd['Year']<2019)]
street_light_reports_pd

## Delete extra columns
#del street_light_reports_pd["Ward", "Police District"]
street_light_reports_pd = street_light_reports_pd.drop(["Ward", "Police District"], axis=1)
street_light_reports_pd.head()

## Question 1 - Which Chicago neighborhoods have the most street light outages reported between 2014-2018?

In [ ]:
#Create Community Area Count Variable
community_area_counts = street_light_reports_pd.groupby(["Community Area"])["Creation Date"].nunique()

#Display Summary of Community Area Counts in Data Frame
community_area_summary_table = pd.DataFrame({"Total Street Light Outage Reportings": community_area_counts})
community_area_summary_table

In [ ]:
#Display Summary of Community Area Names and Corresposonding Number in Data Frame
community_area_names=community_area_names_pd[['COMMUNITY','AREA_NUMBE']]
community_area_names

In [ ]:
# Merge Dataframes to get Street Light Outage Reports and Community Names in One Dataframe
community_data_merge = pd.merge(community_area_summary_table,community_area_names,left_on='Community Area', right_on='AREA_NUMBE', how="right")

#Sort Merged Dataframe by Highest Street Light Outages to Find Community With Highest Street Outages
community_sort = community_data_merge.sort_values(by='Total Street Light Outage Reportings', ascending=False)

# Display the Top 10 Neighborhoods With Most Street Light Outages 2014 to 2018
community_sort.head(10)

In [ ]:
#Display Bar Chart to compare community area street light outages between 2014-2018
community_chart = community_sort.plot.bar(y='Total Street Light Outage Reportings', x='COMMUNITY', rot=90, figsize=(20,3))
plt.savefig("Charts/Street_Light_Outage_by_Community")

In [ ]:
# Filter Street Light Outage Totals by Year 
year_2014 = street_light_reports_pd[street_light_reports_pd['Year']==2014]
year_2015 = street_light_reports_pd[street_light_reports_pd['Year']==2015]
year_2016 = street_light_reports_pd[street_light_reports_pd['Year']==2016]
year_2017 = street_light_reports_pd[street_light_reports_pd['Year']==2017]
year_2018 = street_light_reports_pd[street_light_reports_pd['Year']==2018]

In [ ]:
#Filter Street Light Counts by 2014 
counts_by_year_2014 = year_2014.groupby(["Community Area","Year"])["Creation Date"].nunique()

#Display 2014 Summary of Street Light Outages by Year in Data Frame
counts_by_year_2014_summary_table = pd.DataFrame({ "# of Outages in 2014": counts_by_year_2014})

#Sort Merged Dataframe by Highest Street Light Outages to Find Community With Highest Street Outages
community_sort_2014 = counts_by_year_2014_summary_table.sort_values(by='# of Outages in 2014', ascending=False)

# Display the Top 5 Neighborhoods With Most Street Light Outages in 2014
community_sort_2014.head(5)

In [ ]:
#Filter Street Light Counts by 2015 
counts_by_year_2015 = year_2015.groupby(["Community Area","Year"])["Creation Date"].nunique()

#Display 2015 Summary of Street Light Outages by Year in Data Frame
counts_by_year_2015_summary_table = pd.DataFrame({"# of Outages in 2015": counts_by_year_2015})

#Sort Merged Dataframe by Highest Street Light Outages to Find Community With Highest Street Outages
community_sort_2015 = counts_by_year_2015_summary_table.sort_values(by='# of Outages in 2015', ascending=False)

# Display the Top 5 Neighborhoods With Most Street Light Outages in 2015
community_sort_2015.head(5)

In [ ]:
#Filter Street Light Counts by 2016 
counts_by_year_2016 = year_2016.groupby(["Community Area","Year"])["Creation Date"].nunique()

#Display 2014 Summary of Street Light Outages by Year in Data Frame
counts_by_year_2016_summary_table = pd.DataFrame({"# of Outages in 2016": counts_by_year_2016})

#Sort Merged Dataframe by Highest Street Light Outages to Find Community With Highest Street Outages
community_sort_2016 = counts_by_year_2016_summary_table.sort_values(by='# of Outages in 2016', ascending=False)

# Display the Top 5 Neighborhoods With Most Street Light Outages in 2016
community_sort_2016.head(5)

In [ ]:
#Filter Street Light Counts by 2017 
counts_by_year_2017 = year_2017.groupby(["Community Area","Year"])["Creation Date"].nunique()

#Display 2017 Summary of Street Light Outages by Year in Data Frame
counts_by_year_2017_summary_table = pd.DataFrame({"# of Outages in 2017": counts_by_year_2017})

#Sort Merged Dataframe by Highest Street Light Outages to Find Community With Highest Street Outages
community_sort_2017 = counts_by_year_2017_summary_table.sort_values(by='# of Outages in 2017', ascending=False)

# Display the Top 5 Neighborhoods With Most Street Light Outages in 2017
community_sort_2017.head(5)

In [ ]:
#Filter Street Light Counts by 2018
counts_by_year_2018 = year_2018.groupby(["Community Area","Year"])["Creation Date"].nunique()

#Display 2018 Summary of Street Light Outages by Year in Data Frame
counts_by_year_2018_summary_table = pd.DataFrame({"# of Outages in 2018": counts_by_year_2018})

#Sort Merged Dataframe by Highest Street Light Outages to Find Community With Highest Street Outages
community_sort_2018 = counts_by_year_2018_summary_table.sort_values(by='# of Outages in 2018', ascending=False)

# Display the Top 10 Neighborhoods With Most Street Light Outages in 2018
community_sort_2018.head(5)

## Question 2 - What address has the most street light outage reports? Which year had the most street light outage reports?

## Question 3 - Average time it takes for outages to get fixed per neighborhood?

## Question 4 - What are the crime rates/reports in neighborhoods with street light outages?